#### Medidas de centralidade
- Media (afetada por outliers)
- Mediana
- Moda ( valor mais frequente na amostra

#### Medidas de propagação
- DP - Mede a variabilidade dos dados (sum(yi -ym)^2/(n-1))^0.5
- Quanto maior o desvio maior, pensar em estoque de segurança para atender a demanda dos clientes

#### Conceitos

- Outliers - observaçõe que estão distante da média ( longe da mioria das observações
- 1º quartil - valor aos 25 % da amostra ordenada separa os 25% de dados inferiores
- 2º quartil - mediana
- 3 º quartil - valor aos 75 % da amostra ordenada
- Percentil indica que há x% de dados inferiores Lp= (n-1)*p/100 +1

In [22]:
# Demanda do suco em 500 dias 
import numpy as np
import statistics
suco_apple = np.random.uniform(2,500, 1000).round()
print(f"A média é",suco_apple.mean())
print(f"A médiana é",statistics.median(suco_apple))
print(f"A moda é",statistics.mode(suco_apple))
print(f"O desvio é", statistics.stdev(suco_apple))
print(f"O range é ", max(suco_apple)-min(suco_apple))
print(f"A variância  é", statistics.variance(suco_apple))
print(f"50 % dos dados são inferiores à", np.percentile(suco_apple,50))

A média é 254.612
A médiana é 254.0
A moda é 433.0
O desvio é 142.54754034095873
O range é  495.0
A variância  é 20319.80125725726
50 % dos dados são inferiores à 254.0


#### Correlations

In [24]:
import pandas as pd

In [74]:
class Documento:
    def __init__(self, name):
        self.name =name
        self.df = {}
    def carregar_dados(self):
        self.df = pd.read_csv(self.name)
        return self.df
    def limpar_dados(self):
        return self.df.dropna(axis = 0)
    def corr(self):
        return self.df.iloc[:, [12,13]].corr()
        

In [75]:
doc = Documento('cars.csv')
don = doc.carregar_dados()
clean = doc.limpar_dados()
cor = doc.corr()

In [76]:
print(cor)

                        horsepower  city_miles_per_galloon
horsepower                1.000000               -0.666256
city_miles_per_galloon   -0.666256                1.000000


In [66]:
don = doc.carregar_dados()

In [67]:
clean = doc.limpar_dados()

In [69]:
clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 387 entries, 0 to 403
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      387 non-null    object 
 1   sports_car                387 non-null    bool   
 2   suv                       387 non-null    bool   
 3   wagon                     387 non-null    bool   
 4   minivan                   387 non-null    bool   
 5   pickup                    387 non-null    bool   
 6   all_wheel                 387 non-null    bool   
 7   rear_wheel                387 non-null    bool   
 8   Price                     387 non-null    int64  
 9   Dealer_Cost               387 non-null    int64  
 10  Engine_size               387 non-null    float64
 11  cylenders                 387 non-null    int64  
 12  horsepower                387 non-null    int64  
 13  city_miles_per_galloon    387 non-null    float64
 14  highway_mi

In [33]:
doc.name

'cars.csv'

In [25]:
cars = pd.read_csv('cars.csv')

In [26]:
cars.head()

,name,sports_car,suv,wagon,minivan,pickup,all_wheel,rear_wheel,Price,Dealer_Cost,Engine_size,cylenders,horsepower,city_miles_per_galloon,highway_miles_per_Gallon,weight,base_wheeel,length,width
0,Chevrolet Aveo 4dr,False,False,False,False,False,False,False,11690,10965,1.6,4,103,28.0,34.0,2370.0,98.0,167.0,66.0
1,Chevrolet Aveo LS 4dr hatch,False,False,False,False,False,False,False,12585,11802,1.6,4,103,28.0,34.0,2348.0,98.0,153.0,66.0
2,Chevrolet Cavalier 2dr,False,False,False,False,False,False,False,14610,13697,2.2,4,140,26.0,37.0,2617.0,104.0,183.0,69.0
3,Chevrolet Cavalier 4dr,False,False,False,False,False,False,False,14810,13884,2.2,4,140,26.0,37.0,2676.0,104.0,183.0,68.0
4,Chevrolet Cavalier LS 2dr,False,False,False,False,False,False,False,16385,15357,2.2,4,140,26.0,37.0,2617.0,104.0,183.0,69.0


In [27]:
cars.iloc[:, [12,13]].corr()

,horsepower,city_miles_per_galloon
horsepower,1.000000,-0.666256
city_miles_per_galloon,-0.666256,1.000000


In [28]:
cars.cylenders.value_counts()

 6     190
 4     136
 8      87
 5       7
 12      3
-1       2
 10      2
 3       1
Name: cylenders, dtype: int64

In [30]:
cars.dropna(axis=0, inplace=True)